In [ ]:
# !pip install -r requirements.txt 
import cv2
import numpy as np
from numpy.core.numeric import True_
from numpy.lib.arraysetops import unique
import pandas as pd
# import glob
import os
from pandas import json_normalize
from os import getcwd, path
from yaml import SafeLoader, load
import datetime
import matplotlib.pyplot as plt
import yaml
import shutil
import json

from video_utils_virat import VideoUtils
from video_utils_virat import YoloUtils

%matplotlib inline
cv2.__version__

## Dataset Selection

In [ ]:

# setup
dataset_dir_path = '../datasets/VIRAT/'
image_ext = '.jpg'
video_max_frames = 2000

#video
video_ext = '.mp4'
video_name = 'VIRAT_S_050000_07_001014_001126'
video_name_orig = video_name + video_ext
video_dest_path = './processed/' +  video_name + '/'
video_src_path = dataset_dir_path + 'Videos/Ground/'

using_yml = True

# annotations
saved_csv = video_dest_path + 'df_bbox.csv'


yml_video_name = 'gt_'
yolo_video_name = 'yolo_'

# annotations_path = dataset_dir_path + 'viratannotations/train/' + video_name +'/'
annotations_path = dataset_dir_path + 'viratannotations/validate/' + video_name +'/'
ann_activities_file = annotations_path + video_name + '.activities.yml'
ann_geom_file = annotations_path + video_name + '.geom.yml'
ann_regions_file = annotations_path + video_name + '.regions.yml'
ann_types_file = annotations_path + video_name + '.types.yml'


gt_video_name = yml_video_name + video_name + '.mp4'#video_ext
yolo_video_name = yolo_video_name + video_name + '.mp4'#video_ext

In [ ]:
# Create directory to store new video
if not os.path.exists(video_dest_path):
    os.makedirs(video_dest_path)

if not os.path.exists(saved_csv):
    with open(ann_types_file) as yaml_file:
        yaml_contents = load(yaml_file, Loader=SafeLoader)
    yaml_df = json_normalize(yaml_contents)
    yaml_df
    for col in yaml_df.columns:
        type_name = col.split('.')[-1]
        if not (type_name == 'id1'):
            yaml_df.loc[yaml_df[col] == 1, col] = type_name
    
    yaml_df = yaml_df[yaml_df['types.id1'].notna()].reset_index().dropna(axis=1, how='all')  
    type_df = yaml_df.ffill(axis=1).iloc[:,-1].to_frame(name='category')
    type_df.insert(0, "id", yaml_df['types.id1'])
    type_df


In [ ]:
# using annotations:
print("Loading annotations...")
def add_category_type(row):
  id = row['object_id']
  val = type_df.loc[type_df['id'] == id, 'category'].iloc[0]
  return val


if os.path.exists(saved_csv):
  df_bbox = pd.read_csv(saved_csv)
else:
  with open(ann_geom_file) as yaml_file:
      yaml_contents = load(yaml_file, Loader=SafeLoader)
  yaml_df = json_normalize(yaml_contents)

  df_bbox = yaml_df[['geom.id1','geom.ts0','geom.ts1','geom.g0']].dropna().reset_index()
  df_bbox.rename(columns={'geom.id1': 'object_id', 'geom.ts0': 'frame_id','geom.ts1': 'time_sec', 'geom.g0': 'bbox'}, inplace=True)
  df_bbox['bbox'] = df_bbox['bbox'].str.split()
  df_tmp = pd.DataFrame(df_bbox['bbox'].to_list(), columns = ['bb_left', 'bb_top', 'bb_right', 'bb_bottom'])
  df_bbox = pd.concat([df_bbox, df_tmp], axis=1).drop(columns=['bbox'])

  df_bbox['category'] = df_bbox.apply(lambda row: add_category_type(row), axis=1) 
  df_bbox.drop(columns=['index'], axis=1, inplace=True)
  # df_bbox.set_index['index'] 
  df_bbox.to_csv(saved_csv, index = False)
    

df_bbox.head()

# Add velocities

In [ ]:
# Add velocities for the video

vidcap = cv2.VideoCapture(video_src_path + video_name_orig)
fps = 30
scale = 1/15
if vidcap.isOpened():
    fps = vidcap.get(cv2.CAP_PROP_FPS)  
    print(fps)
    # scale = #TBD


def find_mid(a, b):
  c = (a.astype(float) + b.astype(float))/2
  return c

def find_delta(x):
  dx = x.diff()
  dx[0] = np.nan
  dx.fillna(method='backfill', inplace=True)
  return dx

def find_vel(vx,vy):

  vel = np.sqrt(vx * vx + vy * vy).round(2)
  return vel#, vx, vy

for id in df_bbox['object_id'].unique():
  mask = (df_bbox['object_id']==id)
  sub_df = df_bbox[mask]
  x  = find_mid(sub_df['bb_top'], sub_df['bb_bottom'])
  y  = find_mid(sub_df['bb_left'], sub_df['bb_right'])
  dx = find_delta(x)
  dy = find_delta(y)
  vx = (dx * fps * scale)
  vy = (dy * fps * scale)

  df_bbox.loc[mask, 'x']  = x
  df_bbox.loc[mask, 'y']  = y
  df_bbox.loc[mask, 'dx'] = dx
  df_bbox.loc[mask, 'dy'] = dy
  df_bbox.loc[mask, 'vx'] = (dx * fps * scale)
  df_bbox.loc[mask, 'vy'] = (dy * fps * scale)
  df_bbox.loc[mask, 'vel']  = np.sqrt(vx * vx + vy * vy).round(2)

df_bbox.head()

In [ ]:
from IPython.display import HTML
from IPython.display import display 
from base64 import b64encode

def show_video(file_name, width=640):
  # show resulting deepsort video
  name_split = file_name.split('.')
  ext = name_split[-1]

  # show resulting deepsort video
  file = open(file_name,'rb').read()
  file_type = "video/"+ext
  data_url = "data:" + file_type + ";base64," + b64encode(file).decode()
  return HTML("""
  <video width="{0}" controls>
        <source src="{1}" type="{2}">
  </video>
  """.format(width, data_url, file_type))


In [ ]:
# show_video("test.mp4", width=960)

In [ ]:
from video_utils_virat import YoloUtils
from video_utils_virat import VideoUtils

yUtils = YoloUtils()
vUtils = VideoUtils() 

yolo_weight_file = "../yolo/weights/yolov3.weights"
yolo_cfg_file = "../yolo/darknet/cfg/yolov3.cfg"
yolo_names_file = "../yolo/darknet/data/coco.names"
start_time = 0
video_duration=1#None

# yolo_video_out = os.path.join(video_dest_path,'0-111_yolo_VIRAT_S_050000_07_001014_001126.avi')

df_bbox_vehicle = df_bbox[df_bbox['category'] == 'Vehicle']
yolo_video_out, bbox_yolo = yUtils.YoloOnVideo(yolo_weight_file, yolo_cfg_file, yolo_names_file, video_dest_path, video_src_path + video_name_orig, yolo_video_name, start_time_sec=start_time, duration_sec=video_duration, save_images=False)
gt_video_out, bbox_gt = vUtils.AnnotateVideo(video_dest_path, yolo_video_out, gt_video_name, df_bbox_vehicle, start_time, video_duration)
# gt_video_out, bbox_gt = vUtils.AnnotateVideo(video_dest_path, video_src_path + video_name_orig, gt_video_name, df_bbox, start_time, video_duration)

# show_video(gt_video_out, width=960)

# 
# bbox_gt

In [ ]:
from video_utils_virat import Bbox

def GetMaxCorrelation(bb_gt, pred_bboxes, frame):
    max_iou = 0.0
    max_info = [frame, bb_gt, bb_gt, max_iou]
    found = False

    for bb_pred in pred_bboxes:
        gt = Bbox(bb_gt[0],bb_gt[1],bb_gt[2],bb_gt[3])
        pred = Bbox(bb_pred[0],bb_pred[1],bb_pred[2],bb_pred[3])
        iou = gt.IOU(pred)

        if (max_iou < iou) & (0.5 < iou):
            found = True
            max_iou = iou
            max_info = [frame, bb_gt, bb_pred, iou]

    return found, max_info

df = pd.DataFrame(columns=['frame idx', 'gt bbox', 'pred bbox', 'iou'])
print(f'# of frames in video: {len(bbox_gt)}')
# for i_frame in range(1):#range(len(bbox_gt)): #scan all frames in video
for i_frame in range(len(bbox_gt)): #scan all frames in video
    data = []
    for bb_gt in bbox_gt[i_frame]:
        found, info = GetMaxCorrelation(bb_gt, bbox_yolo[i_frame], i_frame)
        if(found):
            data.append(info)
    
    df_frame = pd.DataFrame(data, columns=['frame idx', 'gt bbox', 'pred bbox', 'iou'])
    df = pd.concat([df, df_frame], ignore_index=True)



mPA = df['iou'].sum()/len(df)
print(mPA)
df.head(30)


In [ ]:
# add IOU to video
from video_utils_virat import VideoUtils
vUtils = VideoUtils() 

gt_video_out

def AddIouToFrame(img, df_info):
    for index, row in df_info.iterrows():
        bb_gt = row['gt bbox']
        txt_val = "{iou:.4f}".format(iou = row['iou'])
        bb = Bbox(bb_gt[0],bb_gt[1],bb_gt[2],bb_gt[3])
        img = vUtils.PrintText(img, txt_val, bb.center_x, bb.center_y, -10, 10, cv2.FONT_HERSHEY_COMPLEX, 0.75, (255, 255, 255))
    return img


def AddIouToVideo(output_dir, orig_v_full_path, df_info):
    if (not orig_v_full_path):
        raise Exception(f"File not found: {orig_v_full_path}")

    bbox_data = {}
    # Open original video
    video_in = cv2.VideoCapture(orig_v_full_path)
    if video_in.isOpened():
        fps, total_frames, frame_size = vUtils.GetVideoData(video_in)
        count = 0
        # setting CV_CAP_PROP_POS_FRAMES at count
        video_in.set(cv2.CAP_PROP_POS_FRAMES, count)
        full_filename = os.path.join(output_dir,'IOU.mp4')
        if os.path.exists(full_filename):
            os.remove(full_filename)

        fourcc = cv2.VideoWriter_fourcc(*"XVID")
        video_out = cv2.VideoWriter(
            full_filename, fourcc, int(fps), frame_size)

        i = 0
        while (True):
            success, img = video_in.read()
            if(success):
                height, width, layers = img.shape
                image_size = (width, height)

                # All bounding box info for this frame
                # Add all annotations to the frame
                df_img = df_info[df_info['frame idx'] == count]
                img = AddIouToFrame(img, df_img)
                i += 1
                video_out.write(img)
                count += 1

            else:
                break

        video_in.release()  # done with original video
        video_out.release()

        print("Done: Created video: " + full_filename)

    cv2.destroyAllWindows()


AddIouToVideo(video_dest_path, gt_video_out, df)



In [ ]:
# from video_utils_virat import VideoUtils
# vUtils = VideoUtils() 
# vUtils.ChangeVideoFrameRate('video_20190906150731_3.avi','test2.avi', 10)

In [ ]:
# from video_utils_virat import YoloUtils
# yUtils = YoloUtils()
# yolo_weight_file = "../../yolo/darknet/yolov3.weights"
# yolo_cfg_file = "../../yolo/darknet/cfg/yolov3.cfg"
# yolo_names_file = "../../yolo/darknet/data/coco.names"
# yUtils.YoloTheWebcam(yolo_weight_file, yolo_cfg_file, yolo_names_file)

In [ ]:
# import os
# import pyautogui

# # this changes the working directory folder to where this script is in
# # os.chdir(os.path.dirname(os.path.abspath(__file__)))
# screenshot = pyautogui.screenshot()
# print(screenshot)

In [ ]:
# import gi
# gi.require_version("Wnck", "3.0")
# from gi.repository import Wnck

# scr = Wnck.Screen.get_default()
# scr.force_update()
# print(scr.get_active_window().get_name())

In [ ]:
from video_utils_virat import YoloUtils
yUtils = YoloUtils()
yolo_weight_file = "../yolo/weights/yolov3.weights"
yolo_cfg_file = "../yolo/darknet/cfg/yolov3.cfg"
yolo_names_file = "../yolo/darknet/data/coco.names"
yUtils.YoloOnScreen(yolo_weight_file, yolo_cfg_file, yolo_names_file, 0,0,1000,1000)
